In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from datetime import timedelta

from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders_prince
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import read_saved_dropbox_state, get_dropbox_folders_prince
import sys
import os

from amftrack.util.sys import get_dirname, temp_path
import pandas as pd
import ast
from scipy import sparse
import scipy.io as sio
import cv2
import imageio.v2 as imageio
import numpy as np
import scipy.sparse
import os
from time import time
from amftrack.pipeline.functions.image_processing.extract_skel import (
    extract_skel_new_prince,
    run_back_sub,
    bowler_hat,
)

from amftrack.util.sys import get_dates_datetime, get_dirname
import shutil
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from matplotlib import cm
import matplotlib.patches as mpatches
import networkx as nx

In [ ]:
# directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
# directory_targ = directory_project
# directory_targ = directory_targ+'myr/'
directory_targ = directory_project

update_plate_info(directory_targ, local=True,strong_constraint=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:

plates_paper = [
    "52_20220707",
    "56_20220701",
    "59_20201126",
    "69_20201119",
    "76_20210108",
    "81_20220704",
    "84_20220621",
    "94_20201123",
    "102_20201226",
    "152_20201224",
    "792_20210915",
    "1076_20220511",
    "941_20220404",
]
plates_paper += [
    "1014_20220525",
    "1023_20220502",
    "1042_20220523",
    "1045_20220504",
    "1048_20220506",
    "1053_20220413",
    "127_20220705",
    "130_20220707",
    "800_20210928",
]

In [ ]:
from amftrack.pipeline.functions.post_processing.time_plate import *


In [ ]:
get_length_study_zone

In [ ]:
from datetime import datetime
results = {}
spore_info = pd.DataFrame()
for plate in plates_paper:
    time_plate_info = {}

    folders = all_folders.loc[all_folders["unique_id"] == plate].copy()
    medium = folders['medium'].iloc[0]
    strain = folders['strain'].iloc[0]
    
    exp = Experiment(directory_targ)
    exp.load_light(folders)
    for t in range(exp.ts):
        data_t = {}
        date = exp.dates[t]
        for index, f in enumerate([get_num_spores,get_spore_volume]):
            column, result = f(exp, t)

            data_t[column] = result
        data_t["date"] = date
        data_t["Plate"] = plate
        data_t["unique_id"] = plate
        data_t["medium"] = medium
        data_t["strain"] = strain
        
        
        time_plate_info[str(t)] = data_t
    folders_plate = folders
    folders_plate = folders_plate.reset_index()
    folders_plate = folders_plate.sort_values("datetime")

    table = pd.DataFrame(time_plate_info).transpose()
    table = table.sort_values("date")
    table["time_since_begin_h"] = (table["date"]-table.iloc[0]["date"])/ np.timedelta64(
            1, "h"
        )
    spore_info = pd.concat(
        [spore_info.copy(), table.copy()], axis=0, ignore_index=True
    )
    # break

In [ ]:
spore_info.to_csv('spores_rebutal.csv')

In [ ]:
import seaborn as sns
fig,ax = plt.subplots()

sns.scatterplot(spore_info,x="time_since_begin_h",y="spore_volume",hue="unique_id",ax = ax)